In [3]:
import os
import numpy as np
import xarray as xr
import pandas as pd
pd.options.mode.chained_assignment = None
from itertools import chain

from utils import *
from bma import *
from metric import *


wd = '/Users/malavirdee/Documents/CI_2023'
os.chdir(wd)
wd = os.getcwd()

models_dir=os.path.join(wd, 'data/models')
reference_dir=os.path.join(wd, 'data/reference')

In [4]:
models = 'GFDL-ESM4,IPSL-CM6A-LR,MPI-ESM1-2-HR,MRI-ESM2-0'#,UKESM1-0-LL'
model_list =models.split(',')
data = load_mf_dataset(models_dir,models)
reference = xr.open_mfdataset(reference_dir+'/*.nc', engine='netcdf4', chunks={'time': 120})

GFDL-ESM4 2
IPSL-CM6A-LR 2
MPI-ESM1-2-HR 2
MRI-ESM2-0 2


In [5]:
df = pd.DataFrame(dict(time=reference.time))
df['W5E5'] = reference.tas.values
for model in model_list:
    df[model] = data[model].tas.values
df.set_index('time', inplace=True)

df['MMM'] = df[model_list].mean(axis=1)

In [6]:
split_date = df.index[len(df.index)//2]
train = df[0:len(df.index)//2]
test = df[len(df.index)//2:]

threshold = np.quantile(train.W5E5.values, 0.9)
print(f'threshold: {threshold:.2f} K')

threshold: 292.94 K


In [7]:
parameters = {}
model_list_bc = [model + '_bc' for model in model_list]

for i, model in enumerate(model_list):
    a = train.W5E5.mean() - train[model].mean()
    bias_corrected = train[model] + a
    train[model+'_bc'] = bias_corrected
    
    parameters[model] = {"a": a}
    
train['MMM_bc'] = train[model_list_bc].mean(axis=1)

for i, model in enumerate(model_list):
    bias_corrected = test[model] + parameters[model]['a']
    test.loc[:,model+'_bc'] = bias_corrected
    
test.loc[:,'MMM_bc'] = test[model_list_bc].mean(axis=1)

In [8]:
weights_standard = bma(train, models = model_list_bc, reference = 'W5E5');
weights_threshold = bma_threshold(train, models = model_list_bc, reference = 'W5E5', threshold = threshold);

In [9]:
w_standard = weights_standard.detach().numpy()
w_threshold = weights_threshold.detach().numpy()

for i, model in enumerate(model_list):
    parameters[model].update({"w_standard": w_standard[i]})
    parameters[model].update({"w_threshold": w_threshold[i]})

train['BMA_standard'] = (train[model_list_bc]*w_standard).sum(axis=1)
test['BMA_standard'] = (test[model_list_bc]*w_standard).sum(axis=1)
train['BMA_threshold'] = (train[model_list_bc]*w_threshold).sum(axis=1)
test['BMA_threshold'] = (test[model_list_bc]*w_threshold).sum(axis=1)

In [10]:
windows = [3,7,15,21,28]
model_list_reordered={}

for window in windows:
    model_list_reordered[window] = [model+'_w{}'.format(window) for model in model_list_bc]
    
for window in windows:
    reordereds = reorder(A=train.W5E5, B=[train[model] for model in model_list_bc], window=window)
    for model, reordered in zip(model_list_bc, reordereds):
        train[model+'_w{}'.format(window)] = reordered

ValueError: Multi-dimensional indexing (e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.

In [ ]:
weights_r = {}
for window in windows:
    weights_r[window]=bma_threshold(train, models=model_list_reordered[window], reference='W5E5', threshold=threshold)
    for i, model in enumerate(model_list):
        parameters[model].update({"w_r{}".format(window): weights_r[window].detach().numpy()[i]})

In [ ]:
w_r=[weights_r[i].detach().numpy() for i in windows]

for i, window in enumerate(windows):
    train['BMA_w{}'.format(window)] = (train[model_list_reordered[window]]*w_r[i]).sum(axis=1)
    test['BMA_w{}'.format(window)] = (test[model_list_bc]*w_r[i]).sum(axis=1)

In [ ]:
M_list = ['BMA_w{}'.format(w) for w in windows] + ['BMA_standard', 'BMA_threshold', 'MMM_bc']
model_list_w = list(chain.from_iterable(model_list_reordered.values()))


evaluate = pd.DataFrame(dict(
    train_q90_n = train[train > threshold].count(),
    test_q90_n = test[test > threshold].count(),
    train_rmse = (train[model_list+model_list_bc + model_list_w  + M_list + ['MMM']].subtract(train.W5E5, axis=0)**2).mean()**0.5,
    test_rmse = (test[model_list+model_list_bc + M_list + ['MMM']].subtract(test.W5E5, axis=0)**2).mean()**0.5,
    train_q90_rmse = (train[train[train>threshold].W5E5.notnull()][model_list+model_list_bc + model_list_w + M_list + ['MMM']].subtract(
        train[train[train>threshold].W5E5.notnull()].W5E5, axis=0)**2).mean()**0.5,
    test_q90_rmse = (test[test[test>threshold].W5E5.notnull()][model_list+model_list_bc + M_list + ['MMM']].subtract(
        test[test[test>threshold].W5E5.notnull()].W5E5, axis=0)**2).mean()**0.5,
))
    
evaluate.loc[['W5E5'] + M_list].round(3).sort_values('test_q90_rmse')

,train_q90_n,test_q90_n,train_rmse,test_rmse,train_q90_rmse,test_q90_rmse
BMA_w15,30,50.0,1.271,1.296,0.981,1.574
BMA_w28,19,33.0,1.171,1.210,1.017,1.653
BMA_w21,28,36.0,1.236,1.240,1.114,1.686
MMM_bc,13,11.0,1.346,1.144,1.615,1.699
BMA_standard,6,20.0,1.252,1.177,1.883,1.863
BMA_w7,41,26.0,1.424,1.350,1.117,1.987
BMA_w3,46,34.0,1.587,1.482,0.999,2.161
BMA_threshold,46,23.0,1.554,1.444,1.356,2.210
W5E5,37,19.0,NaN,NaN,NaN,NaN
